In [1]:
print('Inicializando Variáveis do Programa')

#inicializando variáveis

variável_dependente = 'qty_2019'
variáveis_independentes = ['qty_2014', 'c(group)']
confianças = [0.001, 0.01, 0.05]
indicador = 'Artigo Publicado'
ano_início = 2014
best_r2=False
pareamento=False
capes=True
cnpq=True
nenhum=True
ambos=True
preenche_todos_ids = True
verbose = False

Inicializando Variáveis do Programa


In [2]:
print(confianças)

[0.001, 0.01, 0.05]


In [3]:
confianças.sort()

In [4]:
from datetime import datetime
print('Início em: ', datetime.now())

print('Importações Iniciais')

from tkinter import NONE
from Lattes import Lattes
from Carga import Carga
from Database import Database
import psycopg2, pickle
import pandas, os
import statsmodels.api as sm
from urllib.parse import urlencode
from psycopg2.extensions import AsIs
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import Database
import json
import locale
import statsmodels.formula.api as sm2
from itertools import combinations




print('Fim: ', datetime.now())

Início em:  2023-04-24 22:28:37.471127
Importações Iniciais
Fim:  2023-04-24 22:28:40.213197


In [5]:
sql = '''
select * from financiamentos_doutorandos_2014
    left join indicadores_doutorado
        on indicadores_doutorado.id = financiamentos_doutorandos_2014.id
    inner join indicadores_nomes
        on indicadores_nomes.tipo = indicadores_doutorado.indicador_tipo                 
'''

In [6]:
#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

df = pd.read_sql(sql, engine)
if verbose: print(f'df.size: {df.size}')

print('Término em: ', datetime.now())

Carregando indicadores na memória. Início em:  2023-04-24 22:28:40.245311
Término em:  2023-04-24 22:28:51.201780


In [7]:
sql_ids_pareados = '''
select pareado_1 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_2 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_3 as id from public.par_doutorandos_2014_sem_reposicao
'''

In [8]:
print('Carregando indicadores dos pareados na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

pareados = pd.read_sql(sql_ids_pareados, engine)
if verbose: print(f'pareados.size: {pareados.size}')

print('Término em: ', datetime.now())

Carregando indicadores dos pareados na memória. Início em:  2023-04-24 22:28:51.241146
Término em:  2023-04-24 22:28:51.458214


In [9]:
## Normalizando as Tabelas

print('Realizando normalização da tabela. Início em: ', datetime.now())

df['const'] = 1
df = df.loc[:,~df.columns.duplicated()]

df['bool_cnpq'] = df['pagtos'].map(lambda pagtos: 1 if pagtos > 0 else 0)
df['bool_capes'] = df['capes'].map(lambda capes: 1 if capes == True else 0)



#Preenchendo Nan com zeros nos indicadores
df.qty_2012 = df.qty_2012.fillna(0)
df.qty_2013 = df.qty_2013.fillna(0)
df.qty_2014 = df.qty_2014.fillna(0)
df.qty_2015 = df.qty_2015.fillna(0)
df.qty_2016 = df.qty_2016.fillna(0)
df.qty_2017 = df.qty_2017.fillna(0)
df.qty_2018 = df.qty_2018.fillna(0)
df.qty_2019 = df.qty_2019.fillna(0)
df.capes = df.capes.fillna(0)
df.pagtos = df.pagtos.fillna(0)

valor_total_cnpq =  df[['id', 'pagtos']].loc[df['pagtos']>0].drop_duplicates().sum().pagtos
df['porcent_pagos'] = df['pagtos']/valor_total_cnpq


print('Fim em: ', datetime.now())


Realizando normalização da tabela. Início em:  2023-04-24 22:28:51.467537
Fim em:  2023-04-24 22:28:52.705124


In [10]:
df['grupo'] = df.apply(lambda d: 'Ambos' if (d['capes'] == True and d['pagtos']>0) \
                       else 'CAPES' if (d['capes'] == True and d['pagtos']==0) \
                       else 'CNPq' if d['pagtos'] > 0 \
                       else 'Nenhum', axis=1)
dt = df

In [11]:
#Apagando os financiados 
if not capes:
    if verbose: print('Dropping CAPES')
    dt.drop(dt.loc[dt.grupo == 'CAPES'].index, inplace=True)
if not cnpq:
    if verbose: print('Dropping CNPq')
    dt.drop(dt.loc[dt.grupo == 'CNPq'].index, inplace=True)
if not nenhum:
    if verbose: print('Dropping no financed')
    dt.drop(dt.loc[dt.grupo == 'Nenhum'].index, inplace=True)
if not ambos:
    if verbose: print('Dropping Ambos')
    dt.drop(dt.loc[dt.grupo == 'Ambos'].index, inplace=True)

#Apagando os que não foram escolhidos pelo pareamento
if pareamento:   
    if verbose: print('Dropping Não Pareados')
    dt.drop(dt.loc[~dt.id.isin(pareados.id)].loc[dt.pagtos==0].index, inplace=True)

In [12]:
#Função que dá o tempo para terminar
#para chamar a função:   
#    print_time(feitos, número_total, tempo_início, tempo_último, atual)
#    feitos += 1
#    tempo_último = datetime.now()


tempo_início = datetime.now()
tempo_último = datetime.now()
feitos = 0

def print_time(feitos, número_total, tempo_início, tempo_último, atual):
    if not feitos == 0:
        porcentagem_já_feita = (feitos/número_total)
        print_percent = '{0:.{1}f}'.format(porcentagem_já_feita*100, 1)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / feitos
        tempo_restante = (número_total - feitos) * tempo_por_id
        tempo_em_que_vai_acabar = (datetime.now() + tempo_restante).strftime("%d/%m/%y %H:%M:%S")
        print(f'{feitos}/{número_total}. {print_percent}% feitos. Fazendo: {atual}. Acabará em {tempo_em_que_vai_acabar}. O Último demorou: {datetime.now() - tempo_último}.')
    else:
        print(f'{feitos}/{número_total}. Fazendo: {atual}.')
    

In [13]:
def regressão(dt, indicador, variável_dependente, variáveis_independentes):
    
    lista_formulas = []
    dados = dt.loc[dt.nome==indicador]
    
    if best_r2:
        for x in range(1,len(variáveis_independentes)+1):
            for a in combinations(variáveis_independentes,x):
                formula = f'{variável_dependente} ~ ' 
                if not a[0] == '-1':
                    formula = (formula + ' + '.join(a)).replace('+ -1',' -1')
                    lista_formulas.append(formula)
    else:
        lista_formulas.append(variável_dependente + ' ~ ' + ' + '.join(variáveis_independentes).replace('+ -1',' -1'))
        
    if verbose: print(lista_formulas)
    
    list_results = []
    for formula in lista_formulas:
    
        try:
            # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
            reg_ajustado = sm2.ols(formula, data = dados)
            reg = reg_ajustado.fit()
            # mostrando as estatísticas do modelo
            if verbose: print('\nCom constante:\n',reg.summary2())  

            #guardando a regressão na memória

            result = {
                    'Indicador': indicador,
                    'Erro': False,
                    'Prob (F-statistic)':reg.f_pvalue, 
                    'Parâmetros': reg.params, 
                    "P>|t|":reg.pvalues, 
                    'Standard Error': reg.bse, 
                    "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                    'Number of observations n.': reg.nobs, 
                    "R2 ajustado": reg.rsquared_adj, 
                    "Modelo":reg.model,                }
        except:
            result = {
                    'Indicador': indicador,
                    'Erro': True,
                    'Prob (F-statistic)':None, 
                    'Parâmetros': None, 
                    "P>|t|":None, 
                    'Standard Error': None, 
                    "Covariância entre os pasrâmetros": None, 
                    'Number of observations n.': None, 
                    "R2 ajustado": None, 
                    "Modelo": None,                }
    
        list_results.append(result)
        
    maiorR = max(list_results, key=lambda x:x['R2 ajustado'])
        
    return maiorR

In [14]:
def regressão_com_confiança (dt, indicador, variável_dependente, variáveis_independentes, confiança = 0.05):
    
    variáveis_independentes = list(variáveis_independentes)
    result = regressão(dt, indicador, variável_dependente, variáveis_independentes)
    if verbose: print(result)
    while result['Erro'] == False and result['P>|t|'].max() > confiança:
        if result['P>|t|'].idxmax() == 'Intercept':
            variáveis_independentes.append('-1')
        else:
            variáveis_independentes.remove(result['P>|t|'].idxmax())
        result = regressão(dt, indicador, variável_dependente, variáveis_independentes)
        if verbose: print(result)
        
        
    if result['Erro'] == False and result['Prob (F-statistic)'] < confiança and result['P>|t|'].max() < confiança:
        result['Confiança'] = confiança
        return result
    else:
        result['Confiança'] = 'Erro'
    return result  
    
    

        
        



In [15]:
def regressão_com_lista_confianças (dt, indicador,
                                    variável_dependente, 
                                    variáveis_independentes, 
                                    confianças =  [0.001, 0.01, 0.05]):
    confianças.sort()
    for confiança in confianças:
        result = regressão_com_confiança (dt, indicador, variável_dependente, variáveis_independentes, confiança)
        if not result['Confiança'] == 'Erro':
            break
    if verbose: print(result)
    return result

In [16]:
def preenche_data_frame_com_zeros(dados, indicador = 'Artigo Publicado', verbose=verbose):

    dados = df.loc[df.nome == indicador]
    lista_ids = df.loc[~df.id.isin(dados.id)].id.unique()
    feitos = 0
    tempo_início = datetime.now()
    tempo_último = datetime.now()

    try:
        indicador_tipo = df.iloc[df.loc[df.nome == indicador].first_valid_index()].indicador_tipo
    except:
        indicador_tipo = None
    try:
        indicador_grupo = df.iloc[df.loc[df.nome == indicador].first_valid_index()].indicador_grupo
    except:
        indicador_grupo = None
    
    if verbose: 
        número_total = len(lista_ids)
        num_feitos = 0
        print('\nPreenchendo com zeros os Ids faltantes. Tamanho: ', número_total)

    for id in lista_ids:
        if verbose: 
            atual = id
            print_time(feitos, número_total, tempo_início, tempo_último, atual)
            feitos += 1
            tempo_último = datetime.now()
           

        row = {
            'id': id,
            'pagtos': 0,
            'capes': False,
            'indicador_tipo': indicador_tipo,
            'indicador_grupo': indicador_grupo,
            'indicador': indicador,
            'qty_2012': 0,
            'qty_2013': 0,
            'qty_2014': 0,
            'qty_2015': 0,
            'qty_2016': 0,
            'qty_2017': 0,
            'qty_2018': 0,
            'qty_2019': 0,
            'tipo': indicador_tipo,
            'nome': indicador,
            'grupo': 'Nenhum',
            'path': None,
            'const': 1,
            'bool_cnpq': 0,
            'bool_capes': 0,
            'porcent_pagos': 0
                }
        dados = dados.append(row, ignore_index=True)

    if verbose: print('Terminado.\n')
    return dados
        

In [17]:
def faz_regressões():
    #inicializando variáveis
    lista_regressões = []
    feitos = 0
    tempo_último = datetime.now()
    print(f'Iniciado em {tempo_início}.')

    #pegando lista de todos os indicadores
    todos_indicadores = dt.nome.unique()

    #iterando sobre cada indicador
    número_total = len(todos_indicadores)
    for indicador in todos_indicadores:

        #Realizando previsão de término
        print_time(feitos, número_total, tempo_início, tempo_último, indicador)
        feitos += 1
        tempo_último = datetime.now()


        if preenche_todos_ids: 
            dados = preenche_data_frame_com_zeros(dt, indicador, verbose = False)
            result = regressão_com_lista_confianças(dados, 
                                                    indicador, 
                                                    variável_dependente, 
                                                    variáveis_independentes
                                                   , confianças)
        else:
            result = regressão_com_lista_confianças(dt, 
                                                    indicador, 
                                                    variável_dependente, 
                                                    variáveis_independentes
                                                   , confianças)
        lista_regressões.append(result)
    
    return lista_regressões


In [18]:
def salva_regressões_csv(lista_regressões):
    regressões = pd.DataFrame(lista_regressões)
    regressões = pd.concat([regressões, 
                            regressões['Parâmetros'].apply(pd.Series), 
                            regressões['P>|t|'].apply(pd.Series), 
                            regressões['Standard Error'].apply(pd.Series)], 
                           axis=1)
    regressões.to_excel(f'Regressões dos Doutorandos {ano_início} com Pareamento {pareamento} e sem Reposição com preenchimento com todos CNPq {cnpq} CAPES {capes} Ambos {ambos} Nenhum {nenhum}.xlsx')

In [19]:
lista_regressões = faz_regressões()

Iniciado em 2023-04-24 22:29:07.796788.
0/130. Fazendo: Participação em Banca Julgadora para Professor Titular.
1/130. 0.8% feitos. Fazendo: Publicação de Capítulo de Livro. Acabará em 25/04/23 04:20:20. O Último demorou: 0:02:41.786624.
2/130. 1.5% feitos. Fazendo: Ministério de Curso de Curta Duração. Acabará em 25/04/23 04:11:14. O Último demorou: 0:02:33.692595.
3/130. 2.3% feitos. Fazendo: Produção - Demais Trabalhos. Acabará em 25/04/23 04:08:17. O Último demorou: 0:02:33.825291.
4/130. 3.1% feitos. Fazendo: Desenvolvimento de Material Didático ou Instrucional. Acabará em 25/04/23 04:14:16. O Último demorou: 0:02:47.588061.
5/130. 3.8% feitos. Fazendo: Livro Publicado ou Organizado. Acabará em 25/04/23 04:35:21. O Último demorou: 0:03:27.955196.
6/130. 4.6% feitos. Fazendo: Mídia Social, websiteou Blog. Acabará em 25/04/23 04:44:11. O Último demorou: 0:03:13.479001.
7/130. 5.4% feitos. Fazendo: Música. Acabará em 25/04/23 04:45:02. O Último demorou: 0:02:55.849530.
8/130. 6.2% fe

65/130. 50.0% feitos. Fazendo: Artigo Aceito para Publicação. Acabará em 25/04/23 03:20:50. O Último demorou: 0:03:15.307439.
66/130. 50.8% feitos. Fazendo: Fim de Estágio. Acabará em 25/04/23 03:23:18. O Último demorou: 0:03:29.848979.
67/130. 51.5% feitos. Fazendo: Fim de Extensão Universitária. Acabará em 25/04/23 03:25:42. O Último demorou: 0:03:29.830689.
68/130. 52.3% feitos. Fazendo: Fim de Outra Atividade Técnico/Científica. Acabará em 25/04/23 03:27:48. O Último demorou: 0:03:22.720518.
69/130. 53.1% feitos. Fazendo: Início de Estágio. Acabará em 25/04/23 03:29:45. O Último demorou: 0:03:19.901071.
70/130. 53.8% feitos. Fazendo: Início de Outra Atividade Técnico/Científica. Acabará em 25/04/23 03:31:56. O Último demorou: 0:03:29.335111.
71/130. 54.6% feitos. Fazendo: Orientação em Andamento de Iniciação Científica. Acabará em 25/04/23 03:33:39. O Último demorou: 0:03:15.952334.
72/130. 55.4% feitos. Fazendo: Participação em Exposição. Acabará em 25/04/23 03:35:09. O Último dem

128/130. 98.5% feitos. Fazendo: Obtenção do Título de MBA. Acabará em 25/04/23 04:17:14. O Último demorou: 0:02:46.219995.
129/130. 99.2% feitos. Fazendo: Desenvolvimento de Topografia de Circuito Integrado. Acabará em 25/04/23 04:17:32. O Último demorou: 0:02:57.910348.


In [20]:
salva_regressões_csv(lista_regressões)